In [40]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
# Need to uncomment the following line in order to download nltk stopwords:
# nltk.download('stopwords')
import re
import spacy

# Import data set:

In [41]:
pd.options.mode.chained_assignment = None

with open('data/dk_asylum_data.json', encoding='utf-8') as inputfile:
    df_full = pd.read_json(inputfile)

# Convert json to csv:
df_full.to_csv('data/dk_asylum_data.csv', encoding='utf-8', index=False)

df_full.shape

(9324, 5)

# Data preprocessing:

## Keep cases of Unknown Homeland (Ukendt hjemland):

In [42]:
df_UH = df_full[df_full.country == "Ukendt hjemland"]

# Drop redundant column:
df_UH.drop('hasText', axis=1, inplace=True)

# Keep column 'text':
df = df_UH[['text']]

df["text"] = df["text"].astype(str)

df.shape

df.head()

,text
201,Nævnet stadfæstede i november 2021 Udl...
428,Nævnet stadfæstede i juni 2021 Udlændi...
464,Nævnet stadfæstede i juni 2021 Udlændi...
901,Nævnet stadfæstede i december 2020 Udl...
1291,Nævnet hjemviste i juni 2020 Udlænding...


## Lower casing:

In [43]:
df["text_lower"] = df["text"].str.lower()
df.head()

,text,text_lower
201,Nævnet stadfæstede i november 2021 Udl...,nævnet stadfæstede i november 2021 udl...
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede i juni 2021 udlændi...
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede i juni 2021 udlændi...
901,Nævnet stadfæstede i december 2020 Udl...,nævnet stadfæstede i december 2020 udl...
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævnet hjemviste i juni 2020 udlænding...


## Punctuation removal:

In [44]:
PUNCTUATION_TO_REMOVE = '–«!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~»●·’“”'


def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCTUATION_TO_REMOVE))


df["text_wo_punct"] = df["text_lower"].apply(
    lambda text: remove_punctuation(text))
df.head()

,text,text_lower,text_wo_punct
201,Nævnet stadfæstede i november 2021 Udl...,nævnet stadfæstede i november 2021 udl...,nævnet stadfæstede i november 2021 udl...
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede i juni 2021 udlændi...,nævnet stadfæstede i juni 2021 udlændi...
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede i juni 2021 udlændi...,nævnet stadfæstede i juni 2021 udlændi...
901,Nævnet stadfæstede i december 2020 Udl...,nævnet stadfæstede i december 2020 udl...,nævnet stadfæstede i december 2020 udl...
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævnet hjemviste i juni 2020 udlænding...,nævnet hjemviste i juni 2020 udlænding...


## Stopwords removal:

In [45]:
stopwords_lower = list(map(lambda x: x.lower(), stopwords.words('danish')))

STOPWORDS_DANISH = set(stopwords_lower)

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if not word in STOPWORDS_DANISH])


df["text_wo_stop"] = df["text_wo_punct"].apply(
    lambda text: remove_stopwords(text))
df.head()

,text,text_lower,text_wo_punct,text_wo_stop
201,Nævnet stadfæstede i november 2021 Udl...,nævnet stadfæstede i november 2021 udl...,nævnet stadfæstede i november 2021 udl...,nævnet stadfæstede november 2021 udlændingesty...
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede i juni 2021 udlændi...,nævnet stadfæstede i juni 2021 udlændi...,nævnet stadfæstede juni 2021 udlændingestyrels...
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede i juni 2021 udlændi...,nævnet stadfæstede i juni 2021 udlændi...,nævnet stadfæstede juni 2021 udlændingestyrels...
901,Nævnet stadfæstede i december 2020 Udl...,nævnet stadfæstede i december 2020 udl...,nævnet stadfæstede i december 2020 udl...,nævnet stadfæstede december 2020 udlændingesty...
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævnet hjemviste i juni 2020 udlænding...,nævnet hjemviste i juni 2020 udlænding...,nævnet hjemviste juni 2020 udlændingestyrelsen...


## Numbers removal:

In [46]:
# Drop the columns which are no longer needed
df.drop(["text_lower", 'text_wo_punct'], axis=1, inplace=True)


def drop_numbers(text):
    text_wo_numbers = re.sub(r'[0-9]+', '', text)
    return text_wo_numbers


df["text_wo_numbers"] = df["text_wo_stop"].apply(
    lambda text: drop_numbers(text))
df.head()

,text,text_wo_stop,text_wo_numbers
201,Nævnet stadfæstede i november 2021 Udl...,nævnet stadfæstede november 2021 udlændingesty...,nævnet stadfæstede november udlændingestyrels...
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni 2021 udlændingestyrels...,nævnet stadfæstede juni udlændingestyrelsens ...
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni 2021 udlændingestyrels...,nævnet stadfæstede juni udlændingestyrelsens ...
901,Nævnet stadfæstede i december 2020 Udl...,nævnet stadfæstede december 2020 udlændingesty...,nævnet stadfæstede december udlændingestyrels...
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævnet hjemviste juni 2020 udlændingestyrelsen...,nævnet hjemviste juni udlændingestyrelsens af...


## Single character words removal:

In [47]:
def drop_single_letter_words(text):
    return ' '.join([w for w in text.split() if len(w) > 1])


df["text_wo_single_letters"] = df["text_wo_numbers"].apply(
    lambda text: drop_single_letter_words(text))
df.head()

,text,text_wo_stop,text_wo_numbers,text_wo_single_letters
201,Nævnet stadfæstede i november 2021 Udl...,nævnet stadfæstede november 2021 udlændingesty...,nævnet stadfæstede november udlændingestyrels...,nævnet stadfæstede november udlændingestyrelse...
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni 2021 udlændingestyrels...,nævnet stadfæstede juni udlændingestyrelsens ...,nævnet stadfæstede juni udlændingestyrelsens a...
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni 2021 udlændingestyrels...,nævnet stadfæstede juni udlændingestyrelsens ...,nævnet stadfæstede juni udlændingestyrelsens a...
901,Nævnet stadfæstede i december 2020 Udl...,nævnet stadfæstede december 2020 udlændingesty...,nævnet stadfæstede december udlændingestyrels...,nævnet stadfæstede december udlændingestyrelse...
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævnet hjemviste juni 2020 udlændingestyrelsen...,nævnet hjemviste juni udlændingestyrelsens af...,nævnet hjemviste juni udlændingestyrelsens afg...


## Tokenize text:

In [48]:
# Drop the columns which are no longer needed
df.drop(["text_wo_stop", 'text_wo_numbers'], axis=1, inplace=True)

nlp = spacy.load("da_core_news_sm")

def tokenize_text(text):
    doc = nlp.tokenizer(text)
    return ' '.join([token.text for token in doc])

df["text_tokenized"] = df["text_wo_single_letters"].apply(lambda text: tokenize_text(text))

df.head()

,text,text_wo_single_letters,text_tokenized
201,Nævnet stadfæstede i november 2021 Udl...,nævnet stadfæstede november udlændingestyrelse...,nævnet stadfæstede november udlændingestyrelse...
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni udlændingestyrelsens a...,nævnet stadfæstede juni udlændingestyrelsens a...
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni udlændingestyrelsens a...,nævnet stadfæstede juni udlændingestyrelsens a...
901,Nævnet stadfæstede i december 2020 Udl...,nævnet stadfæstede december udlændingestyrelse...,nævnet stadfæstede december udlændingestyrelse...
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævnet hjemviste juni udlændingestyrelsens afg...,nævnet hjemviste juni udlændingestyrelsens afg...


## Lemmatization:

In [49]:
def lemmatize_words(text):
    """custom function to lemmatize text"""
    doc = nlp(text)
    # pos_tagged_text = text.pos
    return ' '.join([token.lemma_ for token in doc])


df["text_lemmatized"] = df["text_tokenized"].apply(
    lambda text: lemmatize_words(text))
df.head()

,text,text_wo_single_letters,text_tokenized,text_lemmatized
201,Nævnet stadfæstede i november 2021 Udl...,nævnet stadfæstede november udlændingestyrelse...,nævnet stadfæstede november udlændingestyrelse...,nævn stadfæste november udlændingestyrelse afg...
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni udlændingestyrelsens a...,nævnet stadfæstede juni udlændingestyrelsens a...,nævn stadfæste juni udlændingestyrelse afgørel...
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævnet stadfæstede juni udlændingestyrelsens a...,nævnet stadfæstede juni udlændingestyrelsens a...,nævn stadfæste juni udlændingestyrelse afgørel...
901,Nævnet stadfæstede i december 2020 Udl...,nævnet stadfæstede december udlændingestyrelse...,nævnet stadfæstede december udlændingestyrelse...,nævn stadfæste december udlændingestyrelse afg...
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævnet hjemviste juni udlændingestyrelsens afg...,nævnet hjemviste juni udlændingestyrelsens afg...,nævn hjemviste juni udlændingestyrelse afgørel...


## No-sense words removal:

In [51]:
df.drop(["text_wo_single_letters", 'text_tokenized'], axis=1, inplace=True)

# Using spacy’s pos_ attribute, we can check if a particular token 
# is junk (i.e do not mean anything) through token.pos_ == 'X' and remove them.

def remove_junk(text):
    # Creating a spacy object
    doc=nlp(text)
    # Checking if POS tag is X and printing them
    # print('The junk values are..')
    for token in doc:
        if token.pos_=='X':
            print(token.text)
    # print('After removing junk')
    # Removing the tokens whose POS tag is junk.
    clean_doc=[token for token in doc if not token.pos_=='X']
    
    return clean_doc



df['no_junk'] = df["text_lemmatized"].apply(lambda text: remove_junk(text))

df.head()

oplyse
måned
hjælpe
holland
måned
tale
tag
frigive
følge
påberåbe
sprogtests
in
birth
påberåbe
tilknytte
bevisførelse
ortodoks
oplyse
gengive
statsborgerskab
fremlægge
udstede
måned
bopæl
afgive
række
hense
hense
mobiltelefon
tag
virke
hense
as
most
frieri
måned
bopæl
afgive
række
hense
hense
mobiltelefon
tag
virke
hense
oplyse
oplyse
hense
bo
begrænse
rohingya
familie
foto
udstede
specimen
foto
genindkalde
passe
afgive
afgive
sammenholde
tag
hjemlandhjemyars
as
es
læg
føde
måned
es
læg
oplyse
måned
es
læg
anholde
måned
es
es
læg
angribe
tag
oplyse
føde
fremgå
udlændingelov
følge
gø
oplyse
bo
whatsapp
gø
afgive
tag
tie
måned
halv
afgøre
udstede
læg
bopæl
trække
oplyse
kl
følge
udlændingelov
udlænding
møde
tag
påberåbe
følge
udlændingelov
møde
tag
tag
hense
læg
påberåbe
hense
følge
overdrage
tag
hjemmeside
politirapport
tidlig
oplyse
politirapport
land
politirapport
medlem
oplyse
isil
kl
kl
nævnsmøde
oplyse
følge
oplyse
slave
detaljeringsgrad
asylkort
nævnsmøde
udlændingelov
afslag
afgi

,text,text_lemmatized,no_junk
201,Nævnet stadfæstede i november 2021 Udl...,nævn stadfæste november udlændingestyrelse afg...,"[nævn, stadfæste, november, udlændingestyrelse..."
428,Nævnet stadfæstede i juni 2021 Udlændi...,nævn stadfæste juni udlændingestyrelse afgørel...,"[nævn, stadfæste, juni, udlændingestyrelse, af..."
464,Nævnet stadfæstede i juni 2021 Udlændi...,nævn stadfæste juni udlændingestyrelse afgørel...,"[nævn, stadfæste, juni, udlændingestyrelse, af..."
901,Nævnet stadfæstede i december 2020 Udl...,nævn stadfæste december udlændingestyrelse afg...,"[nævn, stadfæste, december, udlændingestyrelse..."
1291,Nævnet hjemviste i juni 2020 Udlænding...,nævn hjemviste juni udlændingestyrelse afgørel...,"[nævn, hjemviste, juni, udlændingestyrelse, af..."
